# 优矿学习笔记
----

## 1. 基础部分
### I. 开始和截止日期

```
start='2013-01-01'
end='2015-01-01'
```

### II.  基准

```
benchmark='HS300'
benchmark = '000001.XSHE'
benchmark = '399006.ZICN'
```
*获取基准清单*
```
DataAPI.IdxGet()
```

### III.  设置初始资金
```
capital_base = 100000
```

### IV.  设置策略类型和调仓频率
*策略类型，'d'表示日间策略使用日线回测，'m'表示日内策略使用分钟线回测*
```
freq = 'd'
```
*调仓频率，表示执行策略运行条件的时间间隔，若freq = 'd'时间间隔的单位为交易日，若freq = 'm'时间间隔为分钟*
```
refresh_rate = 1
```

### V. 股票池
*最近一个交易日的上证50的成份股*
```
universe = set_universe('SH50')
```
*股票池里是指定的，自定义股票池里的证券，XSHE，表示深交所；XSHG，表示上交所*
```
universe = ['000001.XSHE', '600000.XSHG']
```
*用优矿提供的策略框架下的StockScreener来按因子条件筛选*
*这里筛选的因子factor是股票的PE值，筛选的方法是最大的10支*
```
universe = StockScreener(Factor.PE.nlarge(10))
```
*可以将几种不同的定义股票池的方法结合起来使用（就是个list,当然可以+)*
```
universe = StockScreener(Factor.PE.nlarge(10)) + ['000001.XSHE', '600000.XSHG']
```

### VI. 初始化回测帐户
```
def initialize(account):
    pass
```
*account 可以自己定义各种函数*

### VII. 买卖命令
*每只股票买1手（100股）*
```
def handle_data(account):
    for s in account.universe:
        order(s,100)
```
*每只股票买至持仓1手（100股）*
```
def handle_data(account):
      for s in account.universe:
        order_to(s,100)
```
*每只股票买入价值为虚拟账户当前总价值的10%*
```
def handle_data(account):
    for s in account.universe:
        order_pct(s,0.1)
```
*每只股票买入卖出至价值为虚拟账户当前总价值的10%*
```
def handle_data(account):
    for s in account.universe:
        order_pct_to(s,0.1)
```
*e.g.*
```
# 把上面的合起来就完成了一个最简单的策略，从沪深300中任意挑选一只股票，涨到1.25倍后卖出。
start = '2015-07-01'              # 回测的起止时间是2015年7月1日至2016年7月1日
end= '2016-07-01'
benchmark = 'HS300'						 # 参照标准为沪深300的走势
universe = set_universe('HS300')      # 股票池为沪深300的成份股
capital_base = 100000             # 起始本金为10万元

def initialize(account):           # 初始化了一个全新的帐户
    pass

def handle_data(account):
    for stock in account.universe:         # 股票是股票池中的股票，并且优矿帮你自动剔除了当天停牌退市的股票
        p = account.referencePrice[stock]        # 股票前一天的收盘价
        cost = account.valid_seccost.get(stock, p)  # 股票的平均持仓成本
        if 0 < p < 4:                        # 判断这个价格小于4
            order_pct_to(stock, 0.1)          # 将满足条件的股票买入，总价值占虚拟帐户的10%
        elif p >= cost * 1.25:        # 卖出条件，当p这个价格涨幅到买入价的1.25倍；
            order_to(stock, 0)        # 将满足条件的股票卖到剩余0股，即全部卖出
```

### VIII. [免费数据](https://uqer.io/data/)

- 免费数据的调用方法
```
DataAPI.函数名称
```
- 部分收费数据的调用方法：
```
DataAPI.数据提供商缩写.函数名称
```
- DataAPI缓存加速
```
DataAPI.settings.cache_enabled = True / False
```

## 2. 优矿的策略回测框架（Quartz）
[优矿支持的Python模块](https://uqer.io/help/faq/#支持哪些Python模块)

### 4个下单API
- 买入（卖出）数量为amount的股票symbol，otype = 'market'/ 'limit'（为limit时仅日内策略可用）
```
order(symbol, amount, price = 0., otype = 'market')
```
- 买入（卖出）一定量的股票使得股票symbol交易后的数量为amount
```
order_to(symbol, amount, price = 0., otype = 'market')
```
- 买入（卖出）价值为虚拟账户当前总价值的pct部分的证券symbol，仅限市价单。pct范围为 -1 ~ 1，为正则为买入，为负则为卖出；程序会自动对计算出的下单数量向下取整到最近的整百。
```
order_pct(symbol, pct)
```
- 买入（卖出）证券symbol使得其价值为虚拟账户当前总价值的pct部分，仅限市价单
```
order_pct_to(symbol, pct)
```


### observe
在handle_data()中调用observe函数来将某些自定义的需要记录的信息加到回测结果报告中，即增加了一列自定义需要观测的变量。这一方法仅限日间策略。
```
observe(name, value)
```